In [8]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

In [9]:
spark

In [10]:
csv_df = spark.read.csv("../data/footbalDataset.csv", header=True, inferSchema=True)


In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.db.football_games (
  Round STRING,
  Date STRING,
  Time STRING,
  Team STRING,
  Team_Score STRING,
  Opponent_Score STRING,
  Opponent STRING,
  Home_Score_AET STRING,
  Away_Score_AET STRING,
  Home_Penalties STRING,
  Away_Penalties STRING,
  Team_Points STRING,
  Opponent_Points STRING,
  season STRING,
  Location STRING,
  Country STRING,
  Competition STRING
)
USING iceberg
LOCATION 'spark-warehouse/iceberg/db/football_games'
""")

DataFrame[]

In [12]:
csv_df.write.format("iceberg").mode("append").save("local.db.football_games")


In [13]:
result = spark.sql("""
    SELECT Team, Date, Team_Score
    FROM local.db.football_games
    WHERE Team = 'RACING SANTANDER' AND Date = '31/08/2002'
""").show()

+----------------+----------+----------+
|            Team|      Date|Team_Score|
+----------------+----------+----------+
|RACING SANTANDER|31/08/2002|       0.0|
|RACING SANTANDER|31/08/2002|       0.0|
+----------------+----------+----------+



In [14]:
spark.sql("""
    UPDATE local.db.football_games
    SET Team_Score = '1.0'
    WHERE Team = 'RACING SANTANDER' AND Date = '31/08/2002'
""")

DataFrame[]

In [15]:
result = spark.sql("""
    SELECT Team, Date, Team_Score
    FROM local.db.football_games
    WHERE Team = 'RACING SANTANDER' AND Date = '31/08/2002'
""").show()

+----------------+----------+----------+
|            Team|      Date|Team_Score|
+----------------+----------+----------+
|RACING SANTANDER|31/08/2002|       1.0|
|RACING SANTANDER|31/08/2002|       1.0|
+----------------+----------+----------+



In [16]:
spark.sql("""
    DELETE FROM local.db.football_games
    WHERE Team = 'RAYO VALLECANO' AND Date = '01/09/2002'
""")

DataFrame[]

In [17]:
spark.sql("""
    SELECT *
    FROM local.db.football_games
    WHERE Team = 'RAYO VALLECANO' AND Date = '01/09/2002'
""").show()

+-----+----+----+----+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+-----------+
|Round|Date|Time|Team|Team_Score|Opponent_Score|Opponent|Home_Score_AET|Away_Score_AET|Home_Penalties|Away_Penalties|Team_Points|Opponent_Points|season|Location|Country|Competition|
+-----+----+----+----+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+-----------+
+-----+----+----+----+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+-----------+



In [18]:
spark.sql("""
    INSERT INTO local.db.football_games VALUES (
        'ROUND 2',          -- Round
        '15/09/2002',       -- Date
        '20:00',            -- Time
        'NEW TEAM',         -- Team
        3.0,                -- Team_Score
        1.0,                -- Opponent_Score
        'OLD TEAM',         -- Opponent
        NULL,               -- Home_Score_AET
        NULL,               -- Away_Score_AET
        NULL,               -- Home_Penalties
        NULL,               -- Away_Penalties
        3.0,                -- Team_Points
        0.0,                -- Opponent_Points
        2002,               -- season
        'Home',             -- Location
        'spain',            -- Country
        'primera-division'  -- Competition
    )
""")

DataFrame[]

In [21]:
spark.sql("select * from local.db.football_games where Team = 'NEW TEAM'").show()

+-------+----------+-----+--------+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+----------------+
|  Round|      Date| Time|    Team|Team_Score|Opponent_Score|Opponent|Home_Score_AET|Away_Score_AET|Home_Penalties|Away_Penalties|Team_Points|Opponent_Points|season|Location|Country|     Competition|
+-------+----------+-----+--------+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+----------------+
|ROUND 2|15/09/2002|20:00|NEW TEAM|       3.0|           1.0|OLD TEAM|          NULL|          NULL|          NULL|          NULL|        3.0|            0.0|  2002|    Home|  spain|primera-division|
+-------+----------+-----+--------+----------+--------------+--------+--------------+--------------+--------------+--------------+-----------+---------------+------+--------+-------+----------------+
